In [1]:
%autosave 10
from gensim import models
import pickle

Autosaving every 10 seconds


C:\Users\Druv\AppData\Local\conda\conda\envs\ULL_embed\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# method taken from SentEval Examples
def get_dictionary(sentences, threshold=0):
    words = {}
    for s in sentences:
        for word in s:
            words[word] = words.get(word, 0) + 1

    if threshold > 0:
        newwords = {}
        for word in words:
            if words[word] >= threshold:
                newwords[word] = words[word]
        words = newwords
    words['<s>'] = 1e9 + 4
    words['</s>'] = 1e9 + 3
    words['<p>'] = 1e9 + 2

    sorted_words = sorted(words.items(), key=lambda x: -x[1])  # inverse sort
    id2word = []
    word2id = {}
    for i, (w, _) in enumerate(sorted_words):
        id2word.append(w)
        word2id[w] = i

    return id2word, word2id


# Skip gram word2vec
def get_wordvec(model, word2id):
    '''
    model: gensim trained skipgram model
    word2id: word2index
    '''
    word_vec = {}
    
    for word in word2id:
        if word in model.wv.vocab:
            word_vec[word] = model.wv[word]

    return word_vec

In [5]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
# import SentEval.examples.data as data

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = '/Users/Druv/Documents/Jupiter/ULL/Lab3/SentEval/'
# path to the NLP datasets 
PATH_TO_DATA = '/Users/Druv/Documents/Jupiter/ULL/Lab3/SentEval/data'
# path to SkipGram model
path_to_model = '/Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin'
model = models.Word2Vec.load(path_to_model)

# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval


def prepare(params, samples):
    """
    For Skip gram,
    """
    _, params.word2id = get_dictionary(samples)
    # load glove/word2vec format 
    params.word_vec = get_wordvec(model, params.word2id)
    # dimensionality of Skip embeddings
    params.wvec_dim = 100
    return

def batcher(params, batch):
    """
    For Skip gram
    
    """
    # if a sentence is empty dot is set to be the only token
    batch = [sent if sent != [] else [''] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        # the format of a sentence is a lists of words (tokenized and lowercased)
        for word in sent:
            if word in params.word_vec:
                # [number of words, embedding dimensionality]
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            # [number of words, embedding dimensionality]
            sentvec.append(vec)
        # average of word embeddings for sentence representation
        # [embedding dimansionality]
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)
    # [batch size, embedding dimensionality]
    embeddings = np.vstack(embeddings)
    return embeddings


# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks =  ['Length', 'WordContent', 'Depth']
                      # ['TopConstituents','BigramShift', 'Tense',
                      #  'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion']
                      # 
                      # ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC', 'MRPC', 'SICKEntailment', 'STS14']
                      # ['CR', 'MR', 'MPQA', 'SUBJ', 'SST2', 'SST5', 'TREC', 'MRPC', 'SNLI',
#                         'SICKEntailment', 'SICKRelatedness', 'STSBenchmark', 'ImageCaptionRetrieval',
#                         'STS12', 'STS13', 'STS14', 'STS15', 'STS16',
#                         'Length', 'WordContent', 'Depth', 'TopConstituents','BigramShift', 'Tense',
#                         'SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    
    with open('30_Skipgram_2.pkl','wb') as f:
        pickle.dump(results, f)
    print(results)

2018-05-31 00:42:01,297 : loading Word2Vec object from /Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin
2018-05-31 00:42:01,297 : {'mode': 'rb', 'kw': {}, 'uri': '/Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin'}
2018-05-31 00:42:01,297 : encoding_wrapper: {'errors': 'strict', 'encoding': None, 'mode': 'rb', 'fileobj': <_io.BufferedReader name='/Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin'>}
2018-05-31 00:42:01,890 : loading wv recursively from /Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin.wv.* with mmap=None
2018-05-31 00:42:01,890 : setting ignored attribute vectors_norm to None
2018-05-31 00:42:01,890 : loading vocabulary recursively from /Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin.vocabulary.* with mmap=None
2018-05-31 00:42:01,890 : loading trainables recursively from /Users/Druv/Documents/Jupiter/ULL/Lab3/models/10_30_skip.bin.trainables.* with mmap=None
2018-05-31 00:42:01,890 : setting ignored attribute cu

{'WordContent': {'ntest': 10000, 'acc': 65.18, 'devacc': 64.96, 'ndev': 10000}, 'Depth': {'ntest': 10000, 'acc': 22.41, 'devacc': 22.06, 'ndev': 10000}, 'Length': {'ntest': 9996, 'acc': 21.87, 'devacc': 21.08, 'ndev': 9996}}
